In [18]:
from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline
from sklearn import neighbors

def train_mission_final(support_mission):
    '''
    Train the support mission with all available data. Use the best parameters from the hyper-
    parameter test. 
     input : support_mission ID
     return: Pipeline of Model (include TFIDF and Model) 
    '''
    n_neighbors = 3   
    pipe = Pipeline([('knn', neighbors.KNeighborsClassifier(n_neighbors))])
    pipe.fit(test[0], test[1])
    return pipe

test = make_classification(n_samples=100, n_features=20)
model = train_mission_final(test)

In [19]:
from ibm_watson_machine_learning import APIClient

# Credentials
api_key = "<your API KEY>"
location = 'us-south'

wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

# Client Connection
client = APIClient(wml_credentials)
# Deployment Space ID
space_id = "<your SPACE ID>"
assert client.set.default_space(space_id) == 'SUCCESS'
# SW-Spec
sw_spec_uid = client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [28]:
def publish_model(model):
    '''
    This function is to publish the model to the deployment space.
     model: scikit-learn model to publish
     support_mission: support mission name 
    '''
    model_props = {
        client.repository.ModelMetaNames.NAME: "KNN Model for Mission: ",
        client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid,
        client.repository.ModelMetaNames.INPUT_DATA_SCHEMA: [{'id': "asdasdasd",
                                                              'type': 'list',
                                                              'fields': [{'name': 'text', 'type': 'string'}]
                                                             }]
        }
    # publish model to deployment space
    published_model = client.repository.store_model(model=model, meta_props=model_props)
    model_uid = published_model['metadata']['id']
    return model_uid

In [31]:
def deploy_model(model_uid):
    '''
    This function is to deploy the published model to the deployment 
    space.
     model_uid: model_uid from a published model on the deployment space
    '''
    metadata = {
        client.deployments.ConfigurationMetaNames.NAME: "KNN Model for Mission: ",
        client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
    created_deployment = client.deployments.create(model_uid, meta_props=metadata)
    serving_url = created_deployment['entity']['status']['serving_urls'][0]
    deployment_uid = client.deployments.get_id(created_deployment)
    return serving_url, deployment_uid

In [32]:
published_model = publish_model(model)
serving_url, deployment_uid = deploy_model(published_model)

{"trace":"c022dc449cda9a5de64909d4e7a87e09","errors":[{"code":"unsupported_software_specification_details","message":"Software specification runtime-22.1-py3.9 for model is not supported. Supported software specifications for model are: default_py3.7, default_py3.7_opence, default_py3.8. Retry deployment after saving the model with supported software specification."}]}


--------------------------
Deployment creation failed
--------------------------




WMLClientError: Deployment creation failed. Error: 400. {"trace":"c022dc449cda9a5de64909d4e7a87e09","errors":[{"code":"unsupported_software_specification_details","message":"Software specification runtime-22.1-py3.9 for model is not supported. Supported software specifications for model are: default_py3.7, default_py3.7_opence, default_py3.8. Retry deployment after saving the model with supported software specification."}]}

In [ ]:
### THis is my changhe